# Library

In [1]:
import torch
import copy
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from PIL import Image, ImageOps
cudnn.benchmark = True
plt.ion()   # interactive mode
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Dataset

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.image_filenames = []
        self.transform = transform
        self.target_transform = target_transform

        for f in os.listdir(img_dir):
            try:
                # Lấy nhãn từ tên file, giả sử tên là "5_abc.jpg"
                label = int(f.split('_')[0])
                if 0 <= label <= 9:
                    self.image_filenames.append(f)
            except:
                continue  # Bỏ qua file không hợp lệ

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.img_dir, img_name)

        # Load ảnh và giữ nguyên RGB
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = int(img_name.split('_')[0])  # đảm bảo phù hợp lại lần nữa
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [3]:
for f in os.listdir('E:/data/train'):
    try:
        label = int(f.split('_')[0])
        if label > 9:
            print(f"Lỗi nhãn >9: {f}")
    except:
        print(f"Không đọc được nhãn từ: {f}")

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # tăng khả năng chống nhiễu ánh sáng/màu
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

# Dataset & Dataloader
train_dataset = CustomImageDataset('E:/data/train', transform=data_transforms['train'])
val_dataset = CustomImageDataset('E:/data/val', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
model = models.mobilenet_v2(pretrained=True)

# Không huấn luyện phần đặc trưng (nếu muốn fine-tune toàn bộ thì bỏ đoạn này)
for param in model.features.parameters():
    param.requires_grad = False

# Thay lớp phân loại cuối
model.classifier[1] = nn.Linear(model.last_channel, 10)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

e:\anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, device, num_epochs=25):
    since = time.time()
    best_model_path = 'best_model.pth'  # ✅ File lưu mô hình tốt nhất

    torch.save(model.state_dict(), best_model_path)  # Lưu tạm thời trước
    best_acc = 0.0

    model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # ✅ Lưu mô hình tốt nhất theo val accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), best_model_path)
                print(f'>> SAVED best model (epoch {epoch}, acc {epoch_acc:.4f})')

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # ✅ Khôi phục model tốt nhất
    model.load_state_dict(torch.load(best_model_path))
    return model

In [7]:
model = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    dataloaders=dataloaders,
    dataset_sizes=dataset_sizes,
    device=device,
    num_epochs=10  # bạn có thể tăng số epoch nếu muốn
)

Epoch 0/9
----------
train Loss: 2.2308 Acc: 0.1900
val Loss: 1.9042 Acc: 0.3410
>> SAVED best model (epoch 0, acc 0.3410)

Epoch 1/9
----------
train Loss: 1.9217 Acc: 0.3333
val Loss: 1.6692 Acc: 0.4178
>> SAVED best model (epoch 1, acc 0.4178)

Epoch 2/9
----------
train Loss: 1.7725 Acc: 0.3897
val Loss: 1.5320 Acc: 0.4854
>> SAVED best model (epoch 2, acc 0.4854)

Epoch 3/9
----------
train Loss: 1.6442 Acc: 0.4737
val Loss: 1.4996 Acc: 0.5109
>> SAVED best model (epoch 3, acc 0.5109)

Epoch 4/9
----------
train Loss: 1.6321 Acc: 0.4761
val Loss: 1.4822 Acc: 0.5310
>> SAVED best model (epoch 4, acc 0.5310)

Epoch 5/9
----------


KeyboardInterrupt: 